In [0]:
from sklearn.datasets import load_iris

In [0]:
x = load_iris().data
y = load_iris().target

In [0]:
from sklearn.model_selection import train_test_split

In [0]:
from keras.utils import to_categorical

In [0]:
y = to_categorical(y)

In [0]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.20)

In [0]:
import tensorflow as tf
import numpy as np

In [0]:
with tf.name_scope('input'):
  input_features = tf.placeholder(dtype=tf.float32, shape=[None, x.shape[1]], name='input_features')
  input_labels = tf.placeholder(dtype=tf.float32, shape=[None, y.shape[1]], name='input_labels')
  
  with tf.name_scope('model'):
    weights = tf.Variable(tf.random_normal(shape=[x.shape[1], y.shape[1]]), name='weights')
    biases = tf.Variable(tf.random_normal(shape=[y.shape[1]]))
    linear_model = tf.add(tf.matmul(input_features, weights), biases)
    predictions = tf.nn.softmax(linear_model)
    
with tf.name_scope('training_ops'):
  loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=linear_model, labels=input_labels))
  train_op = tf.train.GradientDescentOptimizer(learning_rate=0.001).minimize(loss)
  tf.summary.scalar(name='loss', tensor=loss)
  
with tf.name_scope('metrics'):
  correct_prediction = tf.cast(tf.equal(tf.argmax(predictions, 1), tf.argmax(input_labels, 1)), tf.float32)
  accuracy_op = tf.reduce_mean(correct_prediction)
  tf.summary.scalar(name='accuracy', tensor=accuracy_op)
  
summary= tf.summary.merge_all()
writer = tf.summary.FileWriter(logdir='./logs', graph=tf.get_default_graph())

In [0]:
init_op = tf.global_variables_initializer()

In [0]:
def next_batch(batch_size, features, labels):
  indices = np.arange(start=0, stop=features.shape[0])
  np.random.shuffle(indices)
  indices = indices[:batch_size]
  return features[indices], labels[indices]

In [44]:
!rm -rf logs/
!rm ngrok*
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip ngrok-stable-linux-amd64.zip

--2018-08-17 04:08:02--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 52.4.95.48, 52.23.126.223, 52.44.53.64, ...
Connecting to bin.equinox.io (bin.equinox.io)|52.4.95.48|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5363700 (5.1M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip’

ngrok-stable-linux- 100%[===================>]   5.11M  8.18MB/s    in 0.6s    

2018-08-17 04:08:03 (8.18 MB/s) - ‘ngrok-stable-linux-amd64.zip’ saved [5363700/5363700]

Archive:  ngrok-stable-linux-amd64.zip
  inflating: ngrok                   


In [0]:
LOG_DIR = './logs'
get_ipython().system_raw('tensorboard --logdir {} --host 0.0.0.0 --port 6006 &'.format(LOG_DIR))

In [0]:
get_ipython().system_raw('./ngrok http 6006 &')

In [47]:
! curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

http://588b7cb7.ngrok.io


In [48]:
with tf.Session() as sess:
  sess.run(init_op)
  
  for epoch in range(10):
    for _ in range(int(x_train.shape[0] / 16)):
      batch_x, batch_y = next_batch(batch_size=16, features=x_train, labels=y_train)
      
      _, train_loss, train_accuracy = sess.run([train_op, loss, accuracy_op], feed_dict={input_features: batch_x, input_labels: batch_y})
      print('Epoch {}, loss : {}, accuracy : {}'.format(epoch, train_loss, train_accuracy))

Epoch 0, loss : 2.37924861907959, accuracy : 0.5
Epoch 0, loss : 2.810126304626465, accuracy : 0.5
Epoch 0, loss : 2.040714740753174, accuracy : 0.5625
Epoch 0, loss : 4.523715019226074, accuracy : 0.3125
Epoch 0, loss : 2.5937390327453613, accuracy : 0.5
Epoch 0, loss : 5.6737213134765625, accuracy : 0.25
Epoch 0, loss : 3.424433708190918, accuracy : 0.4375
Epoch 1, loss : 4.821608066558838, accuracy : 0.375
Epoch 1, loss : 2.7297444343566895, accuracy : 0.375
Epoch 1, loss : 3.93794584274292, accuracy : 0.3125
Epoch 1, loss : 3.768157720565796, accuracy : 0.5
Epoch 1, loss : 2.4240469932556152, accuracy : 0.625
Epoch 1, loss : 3.3783860206604004, accuracy : 0.625
Epoch 1, loss : 2.6802077293395996, accuracy : 0.625
Epoch 2, loss : 3.9290618896484375, accuracy : 0.3125
Epoch 2, loss : 2.1588363647460938, accuracy : 0.5625
Epoch 2, loss : 3.3655076026916504, accuracy : 0.4375
Epoch 2, loss : 2.036034345626831, accuracy : 0.5
Epoch 2, loss : 4.590794086456299, accuracy : 0.4375
Epoch 2,